In [54]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score,f1_score

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [5]:
data = pd.read_csv('../data/MachineLearningRating_v3.txt',sep='|',low_memory=False)

In [6]:
data.head()


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [7]:
data.dtypes

UnderwrittenCoverID           int64
PolicyID                      int64
TransactionMonth             object
IsVATRegistered                bool
Citizenship                  object
LegalType                    object
Title                        object
Language                     object
Bank                         object
AccountType                  object
MaritalStatus                object
Gender                       object
Country                      object
Province                     object
PostalCode                    int64
MainCrestaZone               object
SubCrestaZone                object
ItemType                     object
mmcode                      float64
VehicleType                  object
RegistrationYear              int64
make                         object
Model                        object
Cylinders                   float64
cubiccapacity               float64
kilowatts                   float64
bodytype                     object
NumberOfDoors               

In [8]:
data.count()


UnderwrittenCoverID         1000098
PolicyID                    1000098
TransactionMonth            1000098
IsVATRegistered             1000098
Citizenship                 1000098
LegalType                   1000098
Title                       1000098
Language                    1000098
Bank                         854137
AccountType                  959866
MaritalStatus                991839
Gender                       990562
Country                     1000098
Province                    1000098
PostalCode                  1000098
MainCrestaZone              1000098
SubCrestaZone               1000098
ItemType                    1000098
mmcode                       999546
VehicleType                  999546
RegistrationYear            1000098
make                         999546
Model                        999546
Cylinders                    999546
cubiccapacity                999546
kilowatts                    999546
bodytype                     999546
NumberOfDoors               

In [9]:
data.isnull().sum()

UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                         145961
AccountType                   40232
MaritalStatus                  8259
Gender                         9536
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                          552
VehicleType                     552
RegistrationYear                  0
make                            552
Model                           552
Cylinders                       552
cubiccapacity                   552
kilowatts                       552
bodytype                        552
NumberOfDoors               

In [10]:
# Basic statistical details
data.describe()

,UnderwrittenCoverID,PolicyID,PostalCode,mmcode,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,CustomValueEstimate,NumberOfVehiclesInFleet,SumInsured,CalculatedPremiumPerTerm,TotalPremium,TotalClaims
count,1.000098e+06,1.000098e+06,1.000098e+06,9.995460e+05,1.000098e+06,999546.000000,999546.000000,999546.000000,999546.000000,2.204560e+05,0.0,1.000098e+06,1.000098e+06,1.000098e+06,1.000098e+06
mean,1.048175e+05,7.956682e+03,3.020601e+03,5.487770e+07,2.010225e+03,4.046642,2466.743258,97.207919,4.019250,2.255311e+05,NaN,6.041727e+05,1.178757e+02,6.190550e+01,6.486119e+01
std,6.329371e+04,5.290039e+03,2.649854e+03,1.360381e+07,3.261391e+00,0.294020,442.800640,19.393256,0.468314,5.645157e+05,NaN,1.508332e+06,3.997017e+02,2.302845e+02,2.384075e+03
min,1.000000e+00,1.400000e+01,1.000000e+00,4.041200e+06,1.987000e+03,0.000000,0.000000,0.000000,0.000000,2.000000e+04,NaN,1.000000e-02,0.000000e+00,-7.825768e+02,-1.200241e+04
25%,5.514300e+04,4.500000e+03,8.270000e+02,6.005692e+07,2.008000e+03,4.000000,2237.000000,75.000000,4.000000,1.350000e+05,NaN,5.000000e+03,3.224800e+00,0.000000e+00,0.000000e+00
50%,9.408300e+04,7.071000e+03,2.000000e+03,6.005842e+07,2.011000e+03,4.000000,2694.000000,111.000000,4.000000,2.200000e+05,NaN,7.500000e+03,8.436900e+00,2.178333e+00,0.000000e+00
75%,1.391900e+05,1.107700e+04,4.180000e+03,6.005842e+07,2.013000e+03,4.000000,2694.000000,111.000000,4.000000,2.800000e+05,NaN,2.500000e+05,9.000000e+01,2.192982e+01,0.000000e+00
max,3.011750e+05,2.324600e+04,9.870000e+03,6.506535e+07,2.015000e+03,10.000000,12880.000000,309.000000,6.000000,2.655000e+07,NaN,1.263620e+07,7.442217e+04,6.528260e+04,3.930921e+05


In [11]:
# Removing duplicate rows.
print('Duplicate Rows Count : ', data.duplicated().sum())
data=data.drop_duplicates(keep="first")

Duplicate Rows Count :  0


In [12]:
data.shape

(1000098, 52)

In [38]:
# Handling Missing Data
# Separate numeric and non-numeric columns
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns

categorical_features = data.select_dtypes(include=['object', 'category']).columns
categorical_features


Index(['TransactionMonth', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'VehicleType', 'make',
       'Model', 'bodytype', 'VehicleIntroDate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'TermFrequency',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType'],
      dtype='object')

In [37]:
numeric_features

Index(['UnderwrittenCoverID', 'PolicyID', 'PostalCode', 'mmcode',
       'RegistrationYear', 'Cylinders', 'cubiccapacity', 'kilowatts',
       'NumberOfDoors', 'CustomValueEstimate', 'NumberOfVehiclesInFleet',
       'SumInsured', 'CalculatedPremiumPerTerm', 'TotalPremium',
       'TotalClaims'],
      dtype='object')

In [44]:
# Drop columns with all missing values
numeric_features = [col for col in numeric_features if data[col].notna().any()]

In [46]:
# Handle missing values in numeric features
numeric_imputer = SimpleImputer(strategy='mean')
data[numeric_features] = numeric_imputer.fit_transform(data[numeric_features])
data[numeric_features]

,UnderwrittenCoverID,PolicyID,PostalCode,mmcode,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,CustomValueEstimate,SumInsured,CalculatedPremiumPerTerm,TotalPremium,TotalClaims
0,145249.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.000000,0.01,25.0000,21.929825,0.0
1,145249.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.000000,0.01,25.0000,21.929825,0.0
2,145249.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.000000,0.01,25.0000,0.000000,0.0
3,145255.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.000000,119300.00,584.6468,512.848070,0.0
4,145255.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.000000,119300.00,584.6468,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,31520.0,389.0,7493.0,4614100.0,2013.0,4.0,2693.0,110.0,4.0,225531.129942,500000.00,395.8481,347.235175,0.0
1000094,31520.0,389.0,7493.0,4614100.0,2013.0,4.0,2693.0,110.0,4.0,225531.129942,500000.00,395.8481,347.235175,0.0
1000095,31520.0,389.0,7493.0,4614100.0,2013.0,4.0,2693.0,110.0,4.0,225531.129942,500000.00,395.8481,347.235175,0.0
1000096,31519.0,389.0,7493.0,4614100.0,2013.0,4.0,2693.0,110.0,4.0,225531.129942,5000000.00,2.6391,2.315000,0.0


In [47]:
# Impute categorical features
categorical_imputer = SimpleImputer(strategy='most_frequent')
data[categorical_features] = categorical_imputer.fit_transform(data[categorical_features])
data[categorical_features]

,TransactionMonth,Citizenship,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,...,CrossBorder,TermFrequency,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType
0,2015-03-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,No,Monthly,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
1,2015-05-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,No,Monthly,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
2,2015-07-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,No,Monthly,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
3,2015-05-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,No,Monthly,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
4,2015-07-01 00:00:00,,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,...,No,Monthly,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,2015-04-01 00:00:00,ZW,Individual,Mr,English,ABSA Bank,Savings account,Single,Male,South Africa,...,No,Monthly,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant
1000094,2015-06-01 00:00:00,ZW,Individual,Mr,English,ABSA Bank,Savings account,Single,Male,South Africa,...,No,Monthly,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant
1000095,2015-08-01 00:00:00,ZW,Individual,Mr,English,ABSA Bank,Savings account,Single,Male,South Africa,...,No,Monthly,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant
1000096,2014-07-01 00:00:00,ZW,Individual,Mr,English,ABSA Bank,Savings account,Single,Male,South Africa,...,No,Monthly,No excess,Passenger Liability,Passenger Liability,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant


In [48]:
# Feature Engineering (Example: Creating new feature 'PremiumPerClaim')
data['PremiumPerClaim'] = data['TotalPremium'] / (data['TotalClaims'] + 1)


In [50]:
# Encoding Categorical Data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [56]:
# Target variable encoding
label_encoder = LabelEncoder()
data['TotalPremium'] = label_encoder.fit_transform(data['TotalPremium'])
data['TotalPremium']

0          20875
1          20875
2             54
3          35031
4             54
           ...  
1000093    32788
1000094    32788
1000095    32788
1000096    11228
1000097    11228
Name: TotalPremium, Length: 1000098, dtype: int64

In [59]:
# Train-Test Split
X = data.drop(columns='TotalPremium')
y = data['TotalPremium']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y 

0          20875
1          20875
2             54
3          35031
4             54
           ...  
1000093    32788
1000094    32788
1000095    32788
1000096    11228
1000097    11228
Name: TotalPremium, Length: 1000098, dtype: int64

In [62]:
data.columns

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet',
       'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims', 'PremiumPerClaim'],
      dtype='object')

In [64]:
# Models
models = {
    'Linear Regression': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ]),
    'Decision Tree': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(random_state=42))
    ]),
    'Random Forest': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42))
    ]),
    'XGBoost': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(random_state=42))
    ])
}

In [65]:
# Model Evaluation
metrics = {
    'accuracy': accuracy_score,
    'precision': precision_score,
    'recall': recall_score,
    'f1_score': f1_score
}


In [66]:


results = {}

for model_name, model_pipeline in models.items():
    # Train model
    model_pipeline.fit(X_train, y_train)
    
    # Predict
    y_pred = model_pipeline.predict(X_test)
    
    # Evaluate
    results[model_name] = {metric_name: metric_func(y_test, y_pred, average='weighted') 
                           for metric_name, metric_func in metrics.items()}

ValueError: A given column is not a column of the dataframe

In [67]:




# Feature Importance Analysis for Random Forest and XGBoost
rf_model = models['Random Forest']['classifier']
xgb_model = models['XGBoost']['classifier']

rf_feature_importances = rf_model.feature_importances_
xgb_feature_importances = xgb_model.feature_importances_

# Convert feature importances to a DataFrame for better readability
rf_importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': rf_feature_importances}).sort_values(by='Importance', ascending=False)
xgb_importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': xgb_feature_importances}).sort_values(by='Importance', ascending=False)

# Results
print("Model Performance Comparison:\n", pd.DataFrame(results).T)
print("\nRandom Forest Feature Importances:\n", rf_importances_df)
print("\nXGBoost Feature Importances:\n", xgb_importances_df)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [20]:
data['Country'].unique().tolist()

['South Africa']